# Vitessce Widget Tutorial

# Vitessce SpatialQuery plugin usage demo

In [ ]:
#!pip install "vitessce[all]==3.3.0" esbuild_py anndata
!pip install "mlxtend~=0.23.0"
#!pip install -i "https://test.pypi.org/simple/" SpatialQuery
!pip install "SpatialQuery @ git+https://github.com/ShaokunAn/Spatial-Query@main"

In [1]:
from os.path import join
from anndata import read_h5ad
from vitessce import (
    VitessceConfig,
    AnnDataWrapper,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
)
from vitessce.widget_plugins import SpatialQueryPlugin

In [2]:
adata = read_h5ad(join("data", "HBM987_KWLK_254", "secondary_analysis.h5ad"))
zarr_path = join("data", "HBM987_KWLK_254", "secondary_analysis.h5ad.zarr")
adata.write_zarr(zarr_path)

In [3]:
plugin = SpatialQueryPlugin(adata)

In [12]:
vc = VitessceConfig(schema_version="1.0.16", name="Spatial-Query")
dataset = vc.add_dataset("Query results").add_object(AnnDataWrapper(
    adata_path=zarr_path,
    obs_feature_matrix_path="X",
    obs_set_paths=["obs/predicted.ASCT.celltype"],
    obs_set_names=["Cell Type"],
    obs_spots_path="obsm/X_spatial",
    feature_labels_path="var/hugo_symbol",
    coordination_values={
        "featureLabelsType": "Gene symbol",
    }
))

spatial_view = vc.add_view("spatialBeta", dataset=dataset)
lc_view = vc.add_view("layerControllerBeta", dataset=dataset)
sets_view = vc.add_view("obsSets", dataset=dataset)
features_view = vc.add_view("featureList", dataset=dataset)
sq_view = vc.add_view("spatialQuery", dataset=dataset)

obs_set_selection_scope, = vc.add_coordination("obsSetSelection",)
obs_set_selection_scope.set_value(None)

sets_view.use_coordination(obs_set_selection_scope)
sq_view.use_coordination(obs_set_selection_scope)
spatial_view.use_coordination(obs_set_selection_scope)
features_view.use_coordination(obs_set_selection_scope)

vc.link_views([spatial_view, lc_view, sets_view, features_view],
    ["additionalObsSets", "obsSetColor"],
    [plugin.additional_obs_sets, plugin.obs_set_color]
)
vc.link_views_by_dict([spatial_view, lc_view], {
    "spotLayer": CL([
        {
            "obsType": "cell",
            "spatialSpotRadius": 15,
        },
    ])
})

vc.layout((spatial_view | (lc_view / features_view)) / (sets_view | sq_view));

In [13]:
vw = vc.widget(height=900, plugins=[plugin], remount_on_uid_change=False)
vw